# 自動完成 (Completion) API 其他應用展示

In [26]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [5]:
from dotenv import load_dotenv
load_dotenv()

False

In [6]:
import openai
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = "2023-05-15"

COMPLETIONS_MODEL = os.getenv('DEPLOYMENT_NAME','gpt-35-turbo')
API_KEY = os.getenv("OPENAI_API_KEY","<您的 API 鍵值>").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","https://<您的 Azure OpenAI 資源名稱>.openai.azure.com/").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT



## 降低幻覺發生的機會的提示

In [7]:
prompt = """回答我的問題，如果不知道就直接說 "抱歉, 我不知道".\n
Q: 請問誰是 2020 年夏季奧運女子跳高冠軍?\n
A:"""
openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=50,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    engine=COMPLETIONS_MODEL,
    stop="\n"
)["choices"][0]["text"].strip(" \n")

'抱歉, 我不知道。'

## 文字情緒判斷 
嘗試使用 Completion API 來進行自動完成

In [8]:
prompt = """告訴我以下客戶反應是正面還是負面.

Q: 這樣爛的產品都可以賣的出去.

Q: 這玩意真的棒，只是跟廣告有點差距.

Q: 妳家產品用了之後讓我感覺自己像豬八戒照鏡子.
"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=80,
    engine=COMPLETIONS_MODEL,
    stop="\n"
)["choices"][0]["text"].strip(" \n")

'A: 負面, 正面, 負面.'

## 檢查文句中是否包含個人隱私資訊 (PII) 
嘗試使用 gpt-35-turbo 與 gpt-4 方支援的 ChatCompletion API 來進行自動完成

In [9]:


response  = openai.ChatCompletion.create( 
  engine = COMPLETIONS_MODEL ,
  messages = [ 
    {'role': 'user', 'content': '以條列方式找出以下跟個人隱私資訊相關的內容\n\nJohn Doe is a 35-year old man and he lives at 21 Main Street, New York, NY. He is a software engineer and he works at Google. He has a wife named Jane Doe and they have two children'
    
    }
  ],
  temperature = 0,
  max_tokens=400
)

print(response['choices'][0]['message']['content']) 

- John Doe's age
- John Doe's address (21 Main Street, New York, NY)
- John Doe's occupation (software engineer)
- John Doe's employer (Google)
- John Doe's marital status (married to Jane Doe)
- John Doe's number of children (two)
